Hi there! 

This is an attempt at making a detailed and well thought out kernel, hope you gain some insights from it and find it useful! Do upvote and share it if you like it! :)

![](https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSjSMEM7M4zB_VH6H6ny0nwI677pU2BbVh6wA&usqp=CAU)

This kernel has covered 4 topics:

- Basic Introduction
- EDA
- Feature Engineering
- Model Building


The name of the competition is **CommonLit Readability Prize**

The brief description line is **Rate the complexity of literary passages for grades 3-12 classroom use**

# Basic Introduction

### What is this Competition about? 


![](https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQnDADH9cHqgFNYDDuBNJSLL2Izyx3l35B00w&usqp=CAU)


**The curx of this competition if to help students!!**

- In this competition, we are required to build an algorithm to rate the complexity of reading passages for grade 3-12 classroom use.

- To accomplish this, we'll have to pair our machine learning skills with a dataset that includes readers from a wide variety of age groups and a large collection of texts taken from various domains. 

- Winning models will be sure to incorporate text cohesion and semantics.

- If successfull, students will benefit from feedback on the complexity and readability of their work, making it far easier to improve essential reading skills.

### And what is our task?

In technical terms,

Given a ***training.csv*** file in which we will have (among other) 2 columns: excerpt and target, we will have to train Machine Learning model(s) that can approximate the relationship between excerpt and the target.

In simple words, we will have to train a Model which can predict the target value given a text excerpt.

This can be formulated as a **Regression problem** with text

### What data do we have?



#### train.csv / test.csv - the training and testing set

- id - unique ID for excerpt
- url_legal - URL of source
- license - license of source material
- excerpt - text to predict reading ease of
- target - reading ease
- standard_error - measure of spread of scores among multiple raters for each excerpt


 **Note: url_legal, license and standard error are blank in the test set.**

### But how do we know we are doing it correctly or well if we win??

Submissions are scored on the root mean squared error. RMSE is defined as:

![](https://community.qlik.com/legacyfs/online/128958_2016-06-23%2013_45_36-Root%20Mean%20Squared%20Error%20_%20Kaggle.png)


# EDA

In [ ]:
# importing
import os
import re
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns
import itertools
import nltk 
import string
from wordcloud import WordCloud
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')
from nltk.stem import WordNetLemmatizer
from textblob import TextBlob,Word
from collections import Counter
import seaborn as sns
import os
import re
from tqdm.notebook import tqdm

import nltk
from nltk.corpus import stopwords
from collections import defaultdict

#visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns
import plotly 
import plotly.graph_objs as go
import plotly.figure_factory as ff
import plotly.io as pio
from plotly.subplots import make_subplots
import plotly.express as px
from plotly.offline import iplot, init_notebook_mode
import cufflinks as cf
import plotly.figure_factory as ff 
from plotly.offline import iplot
from plotly import tools
colors = px.colors.qualitative.Prism
pio.templates.default = "plotly_white"

import plotly.graph_objs as go
import plotly.figure_factory as ff
import plotly.express as px
from plotly.subplots import make_subplots
from plotly.offline import iplot
from wordcloud import WordCloud
from plotly.offline import iplot

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import Ridge, LinearRegression
from sklearn.metrics import mean_squared_error




import nltk
from nltk import tokenize
from nltk.corpus import wordnet, stopwords
from nltk.stem import WordNetLemmatizer

def cprint(string:str, end="\n"):
    """
    A little utility function for printing and stuff
    """
    _pprint(f"[black]{string}[/black]", end=end)
    
    
# see our files:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
#reading data
train = pd.read_csv('/kaggle/input/commonlitreadabilityprize/train.csv')
test = pd.read_csv('/kaggle/input/commonlitreadabilityprize/test.csv')
train_df = train
test_df = test

**Checking out the first five lines for test and train sets**


In [ ]:
print("The TRAIN set")
print(train.head())
print()
print("The TEST set")
print(test.head())

**Number of Rows and Columns**

In [ ]:
print('Rows and Columns in train dataset:', train.shape)
print('Rows and Columns in test dataset:', test.shape)

**The first thing we are going to check is the distribution of the target feature. It's important to know if the distribution.**

In [ ]:
color_map = ['#eeb977', 'lightgray']
sns.set_palette(sns.color_palette(color_map))
plt.figure(figsize=(8, 8), facecolor='#f6f5f5')
plt.title(f"Target Column Distribution")
sns.histplot(train['target'], stat='density',)
sns.kdeplot(train['target'], color='black')
plt.axvline(train['target'].mean(), color='red', linestyle='--', linewidth=0.8)
min_ylim, max_ylim = plt.ylim()
plt.text(train['target'].mean()*1.05, max_ylim*0.96, 'Mean (μ): {:.2f}'.format(train['target'].mean()))
plt.xlabel("Value")
plt.ylabel("Density")
plt.show()

In [ ]:
color_map = ['#eeb977', 'lightgray']
sns.set_palette(sns.color_palette(color_map))

plt.figure(figsize=(8, 8), facecolor='#f6f5f5')
plt.title(f"Standard Error Column Distribution")
sns.histplot(train['standard_error'], stat='density')
sns.kdeplot(train['standard_error'], color='black')
plt.axvline(train['standard_error'].mean(), color='red', linestyle='--', linewidth=0.8)
min_ylim, max_ylim = plt.ylim()
plt.text(train['standard_error'].mean()*1.05, max_ylim*0.96, 'Mean (μ): {:.2f}'.format(train['standard_error'].mean()))
plt.xlabel("Value")
plt.ylabel("Density")
plt.show()

### Observations : 

- In general, the standard error is lowest when the readability is around -1. It tends to get higher for more extreme values of readability.
- There is one data point with 0 readability and 0 standard error.


## License and Target

In [ ]:
train['license_type_cnt'] = train.groupby('license').transform('count')['id']
license_data = train[train['license_type_cnt'] >= 10]

fig, ax = plt.subplots(figsize=(12, 8), facecolor='#f6f5f5')
sns.pointplot(
    data=license_data,
    x='license',
    y='target',
    ci='sd',
    join=False
)

plt.show()

### Observations

- Different licenses have different average readability, but the standard deviation is quite high.

Note that in test data, the license field is always blank.

### NGRAMS

In [ ]:
def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

lemmatizer = WordNetLemmatizer()
def clean_text(text):
    text = re.sub('[^A-Za-z0-9]+', ' ', text.lower())
    words = nltk.word_tokenize(text)
    tagged = nltk.pos_tag(words)
    words = [lemmatizer.lemmatize(word, get_wordnet_pos(pos)) for word, pos in tagged]
    words = [word for word in words if word not in stopwords.words('english')]
    return words

def get_ngrams(words, n):
    return [tuple(words[i:i+n]) for i in range(len(words)-n+1)]

corpus = []
for text, target in train[['excerpt', 'target']].itertuples(index=False):
    sentences = []
    for sentence in tokenize.sent_tokenize(text):
        words = clean_text(sentence)
        unigrams = get_ngrams(words, n=1)
        bigrams = get_ngrams(words, n=2)
        trigrams = get_ngrams(words, n=3)
        sentences.append(words)
    corpus.append({
        'target' : target,
        'text' : text,
        'sentences' : sentences,
        'unigrams' : unigrams,
        'bigrams' : bigrams,
        'trigrams' : trigrams,
    })

corpus = sorted(corpus, key=lambda x: x['target'])

In [ ]:
def plot_grams_target(gram_type):
    gram_cnt = defaultdict(lambda: 0)
    gram_sum = defaultdict(lambda: 0.)
    gram_avg = {}

    for datapoint in corpus:
        for gram in datapoint[gram_type]:
            gram_cnt[gram] += 1
            gram_sum[gram] += datapoint['target']

    for gram in gram_cnt:
        if gram_cnt[gram] >= 5:
            gram_avg[gram] = gram_sum[gram] / gram_cnt[gram]
    
    fig, ax = plt.subplots(1, 2, figsize=(12, 8), facecolor='#f6f5f5')
    
    top_lowest = sorted(gram_avg.items(), key=lambda x: x[1])[:10]
    ngrams, avg_target = zip(*top_lowest)
    ax[0].bar(
        range(len(ngrams)),
        avg_target
    )

    ax[0].set_title(f'{gram_type} with lowest readability')
    ax[0].set_xlabel(gram_type)
    ax[0].set_ylabel('Average readability')
    ax[0].set_xticks(range(len(ngrams)))
    ax[0].set_xticklabels([' '.join(x) for x in ngrams], rotation='vertical')
    
    top_highest = sorted(gram_avg.items(), key=lambda x: x[1])[-10:]
    ngrams, avg_target = zip(*top_highest)
    ax[1].bar(
        range(len(ngrams)),
        avg_target
    )

    ax[1].set_title(f'{gram_type} with highest readability')
    ax[1].set_xlabel(gram_type)
    ax[1].set_ylabel('Average readability')
    ax[1].set_xticks(range(len(ngrams)))
    ax[1].set_xticklabels([' '.join(x) for x in ngrams], rotation='vertical')

    plt.show()

In [ ]:
plot_grams_target('unigrams')

In [ ]:
plot_grams_target('bigrams')

### Target and sentence length

In [ ]:
top_lowest = corpus[:500]
lowest_target_sentence_lengths = [ \
    np.mean([len(sentence) for sentence in datapoint['sentences']]) \
    for datapoint in top_lowest \
]

top_highest = corpus[-500:]
highest_target_sentence_lengths = [ \
    np.mean([len(sentence) for sentence in datapoint['sentences']]) \
    for datapoint in top_highest \
]

top_lowest_mean = np.mean(lowest_target_sentence_lengths)
top_lowest_std = np.std(lowest_target_sentence_lengths)
top_highest_mean = np.mean(highest_target_sentence_lengths)
top_highest_std = np.std(highest_target_sentence_lengths)

fig, ax = plt.subplots(figsize=(8, 10), facecolor='#f6f5f5')
ax.errorbar(
    x=[0, 1],
    y=[top_lowest_mean, top_highest_mean],
    yerr=[top_lowest_std, top_highest_std],
    fmt='o'
)

ax.set_title('Average sentence length and Readability')
ax.set_ylabel('Sentence length')
ax.set_xticks([0, 1])
ax.set_xticklabels(['Top lowest readability', 'Top highest readability'])

plt.show()

In [ ]:
!pip install wordfreq

from wordfreq import word_frequency
lowest_target_word_freq = [
    [word_frequency(word[0], 'en') for word in datapoint['unigrams']]
    for datapoint in top_lowest
]
highest_target_word_freq = [
    [word_frequency(word[0], 'en') for word in datapoint['unigrams']]
    for datapoint in top_highest
]

In [ ]:
lowest_min_freq = [np.min(datapoint) for datapoint in lowest_target_word_freq]
highest_min_freq = [np.min(datapoint) for datapoint in highest_target_word_freq]

fig, ax = plt.subplots(figsize=(6, 8), facecolor='#f6f5f5')
ax.errorbar(
    x=[0, 1],
    y=[np.mean(lowest_min_freq), np.mean(highest_min_freq)],
    yerr=[np.std(lowest_min_freq), np.std(highest_min_freq)],
    fmt='o'
)

ax.set_ylabel('Min word frequency in general English')
ax.set_xticks([0, 1])
ax.set_xticklabels(['Top lowest readability', 'Top highest readability'])

plt.show()

### Observations:
- Hard-to-read texts often contain some less-usual words.
- Texts with shorter sentence lengths are often easier to read.

### Character Frequency

In [ ]:
char_freq_count = train['excerpt'].str.len()

plt.figure(figsize=(8, 8), facecolor='#f6f5f5')
plt.title(f"Character Frequency Count")
sns.histplot(char_freq_count, stat='density')
sns.kdeplot(char_freq_count, color='purple')
plt.axvline(np.mean(char_freq_count), color='red', linestyle='--', linewidth=0.8)
min_ylim, max_ylim = plt.ylim()
plt.text(np.mean(char_freq_count)*0.64, max_ylim*0.96, 'Mean (μ): {:.2f}'.format(np.mean(char_freq_count)))
plt.xlabel("Count")
plt.ylabel("Density")
plt.show()

### Word Count Distribution

In [ ]:
word_count = train['excerpt'].str.split().map(lambda x: len(x))

plt.figure(figsize=(8, 8), facecolor='#f6f5f5')
plt.title(f"Word Count Distribution")
sns.histplot(word_count, stat='density')
sns.kdeplot(word_count, color='purple')
plt.axvline(np.mean(word_count), color='red', linestyle='--', linewidth=0.8)
min_ylim, max_ylim = plt.ylim()
plt.text(np.mean(word_count)*0.86, max_ylim*0.93, 'Mean (μ): {:.2f}'.format(np.mean(word_count)))
plt.xlabel("Word Count")
plt.ylabel("Density")
plt.show()

In [ ]:
unq_word_count = train['excerpt'].apply(lambda x: len(set(str(x).split()))).to_list()

fig = ff.create_distplot([unq_word_count], ['Excerpt Text'],colors = ['#eeb977', 'lightgray'])
fig.update_layout(title_text="Unique Word Count Distribution")
iplot(fig)

In [ ]:
# Generate WordCloud
words = " ".join(train['excerpt'].tolist())
wc = WordCloud(width = 5000, height = 4000, background_color = '#f6f5f5', min_font_size = 10).generate(words)

# Plot it
plt.figure(figsize = (12, 12), facecolor = 'k', edgecolor = 'k' ) 
plt.imshow(wc) 
plt.axis("off") 
plt.tight_layout(pad = 0)

plt.show()

In [ ]:
train['excerpt_length'] = train['excerpt'].apply(lambda x: len(x))
col = 'excerpt_length'
display(pd.DataFrame(train[col].describe()))
sns.histplot(train[col])
plt.show()


print('correlation:')
display(train[['target', 'excerpt_length']].corr())

plt.figure(figsize=(8, 6))
sns.scatterplot(x=train['excerpt_length'], y=train['target'])
sns.regplot(x=train['excerpt_length'], y=train['target'], scatter=False, ci=None, color='tab:blue')
plt.ylabel('reading ease')
plt.show()

In [ ]:
train['word_count'] = train['excerpt'].apply(lambda x: len(x.split()))
col = 'word_count'
display(pd.DataFrame(train[col].describe()))
sns.histplot(train[col])
plt.show()
print('correlation:')
display(train[['target', 'word_count']].corr())

plt.figure(figsize=(8, 6))
sns.scatterplot(x=train['word_count'], y=train['target'])
plt.ylabel('reading ease')
plt.show()

# Modelling
Let's try some basic modelling and then make a submission using that!

In [ ]:
# Split the data roughly
training_data = train
training_file = pd.read_csv("../input/commonlitreadabilityprize/train.csv")
test_file = pd.read_csv("../input/commonlitreadabilityprize/test.csv")
submission = pd.read_csv("../input/commonlitreadabilityprize/sample_submission.csv")
data = training_file[['excerpt', 'target']]
data = data.sample(frac=1).reset_index(drop=True)
excerpt, targets = training_file['excerpt'].values, training_file['target'].values

t_X, v_X = excerpt[:2750], excerpt[2750:]
t_Y, v_Y = targets[:2750], targets[2750:]

print(t_X.shape, v_X.shape)
print(t_Y.shape, v_Y.shape)

In [ ]:
# Make an Sklearn pipeline for this Ridge Regression
backbone_ridge = Ridge(fit_intercept=True, normalize=False)
pipeline_ridge = make_pipeline(
    TfidfVectorizer(binary=True, ngram_range=(1, 1)),
    backbone_ridge
)

# Do training
pipeline_ridge.fit(t_X, t_Y)

# Evaluate the performance on validation set
preds = pipeline_ridge.predict(v_X)
mse_loss = mean_squared_error(v_Y, preds)

print(f"MSE Loss using Ridge and TfIdfVectorizer: {mse_loss}")

In [ ]:
# Make an Sklearn pipeline for this Linear Regression
backbone_linear = LinearRegression(fit_intercept=True, normalize=False)
pipeline_linear = make_pipeline(
    TfidfVectorizer(binary=True, ngram_range=(1, 1)),
    backbone_linear
)

# Do training
pipeline_linear.fit(t_X, t_Y)

# Evaluate the performance on validation set
preds = pipeline_linear.predict(v_X)
mse_loss = mean_squared_error(v_Y, preds)

print(f"MSE Loss using Linear Regression and TfIdfVectorizer: {mse_loss}")

In [ ]:
# Weights for blending later
lin_wgt = 0.2
rig_wgt = 0.8

In [ ]:
# Get the testing file
test = test_file[['id', 'excerpt']]
test_ids = test['id'].tolist()
test_text = test['excerpt'].values

# Do Predictions on testing set
test_preds_ridge = pipeline_ridge.predict(test_text)
test_preds_linear = pipeline_linear.predict(test_text)

# Form a submissions file and save it
submission = pd.DataFrame()
submission['id'] = test_ids
submission['target'] = (test_preds_ridge + test_preds_linear) / 2
submission.to_csv("submission.csv", index=None)
print('Predictions are done!')

### **Notebooks taken help from!**

https://www.kaggle.com/heyytanay/commonlit-eda-understanding-the-competition


https://www.kaggle.com/tungmphung/commonlit-readability-eda/notebook#Target-and-sentence-length


### **If there are any suggesion for the notebook please comment, that would be helpful. Also please upvote if you liked it! Thank you!!**

**Some of my other works:**

**[TPS-APR-21 EDA AND MODEL](https://www.kaggle.com/udbhavpangotra/tps-apr21-eda-model/data)**

**[Heart Attacks Extensive EDA and Visualizations](https://www.kaggle.com/udbhavpangotra/heart-attacks-extensive-eda-and-visualizations)**


**[WHAT ARE PEOPLE WATCHING IN GREAT BIRTAIN](https://www.kaggle.com/udbhavpangotra/what-do-people-use-youtube-for-in-great-britain)**



### Do **drop a comment AND follow me** for more content!!! 